In [ ]:
import pandas as pd
import numpy as np
import utilities
import preprocess
import similarities
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [ ]:
sentence_embeddings =  ['stsb-roberta-large',
                        'all-MiniLM-L6-v2',
                        'all-MiniLM-L12-v2',
                        'all-mpnet-base-v2',
                        'all-distilroberta-v1',
                        'bert-base-nli-mean-tokens',
                        'distiluse-base-multilingual-cased-v1',
                        'distilbert-base-nli-mean-tokens',
                        'multi-qa-mpnet-base-dot-v1']

microsoft/mpnet-base	60.99
nghuyong/ernie-2.0-en	60.73
microsof/deberta-base	60.21
roberta-base	59.63
t5-base	59.21
bert-base-uncased	59.17
distilbert-base-uncased	59.03

In [ ]:
data = 'opp115'
random_state = 1

embedding_method = 'stsb-roberta-large'#'distilbert-base-nli-mean-tokens'
np.random.seed(random_state)
data_paths = {'opp115'   : r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv',
              'ohsumed'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\ohsumed.csv',
              'reuters'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\Reuters21578.csv'}

In [ ]:
df = utilities.read_data(data_paths[data])
X = df['text'].apply(preprocess.preprocess_text)
y = df.drop(['text'], axis=1)

In [ ]:
def calculate_between_class_similarities(col1, col2, X, y):
    
    sims = []
    
    for idx1 in y[y[col1]==1].index:
        for idx2 in y[y[col2]==1].index:
            sims.append(similarities.vector_similarity(X.loc[idx1], X.loc[idx2]))
    
    return sum(sims)/len(sims)     

In [ ]:
def calculate_similarity_matrix(X, y, sim_method='cosine'):
    
    import similarities
    
    sim_df = pd.DataFrame(index=y.columns, columns=y.columns)
    
    for col in y.columns:
    
        indexes = y[y[col]==1].index
        sim_df.loc[col, col] = similarities.calculate_within_class_similarity(X.loc[indexes], 'average')
    
    for col1, col2 in list(combinations(y.columns, 2)):
        sim_df.loc[col1, col2] = calculate_between_class_similarities(col1, col2, X, y)
    
    return sim_df

In [ ]:
X_num = utilities.vectorize_data(X, embedding_method)
X_num = pd.Series([np.squeeze(i) for i in X_num])
sim_df = calculate_similarity_matrix(X_num, y)

In [ ]:
l_sum = 100*np.diag(sim_df).sum()/12 - (sim_df.sum().sum()-np.diag(sim_df).sum())/66
print(f'{l_sum:.2f}')  

In [ ]:
plt.figure(figsize=(7,5))

sns.heatmap(sim_df.fillna(0), annot=True,
xticklabels=sim_df.columns,
yticklabels=sim_df.columns, cmap="rocket_r")

plt.show()

In [ ]:
sample_size = 30

In [ ]:
train_set = []

for col in y.columns:
    
    idxs = y[y[col]==1].index
    tmp_set = X.loc[idxs].sample(sample_size)
    
    for pair in combinations(tmp_set, 2):
        train_set.append(InputExample(texts=list(pair), label=1.0))

In [ ]:
for col1, col2 in combinations(y.columns,2):
    idxs1 = y[y[col1]==1].index
    idxs2 = y[y[col2]==1].index
    tmp_set1 = X.loc[idxs1].sample(int(sample_size/2))
    tmp_set2 = X.loc[idxs2].sample(int(sample_size/2))
    
    for pair in zip(tmp_set1, tmp_set2):
        train_set.append(InputExample(texts=list(pair), label=0.0))

In [ ]:
len(train_set)

In [ ]:
model = SentenceTransformer(embedding_method, device='cuda') # stsb-roberta-large

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)


model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=50)

In [ ]:
def vectorize_data(text, model):
    
    # model = SentenceTransformer(model_name)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    vectors = model.encode(text, convert_to_tensor=False, device=device)
    
    return vectors

In [ ]:
import torch

In [ ]:
X_num_tuned = vectorize_data(X, model)
X_num_tuned = pd.Series([np.squeeze(i) for i in X_num_tuned])
sim_df_after = calculate_similarity_matrix(X_num_tuned, y)

sum_after = 100*np.diag(sim_df_after).sum()/12 - (sim_df_after.sum().sum()-np.diag(sim_df_after).sum())/66
print(f'{sum_after:.2f}')  

In [ ]:
plt.figure(figsize=(7,5))

sns.heatmap(sim_df_after.fillna(0), annot=True,
xticklabels=sim_df.columns,
yticklabels=sim_df.columns, cmap="rocket_r")

plt.show()